In [4]:
%pip install opencv-python
%pip install ipynb
%pip install python-math
%pip install matplotlib
%pip install numpy
%pip install apriltag

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.Collecting apriltag
  Using cached apriltag-0.0.16.tar.gz (201 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build apriltag



  error: subprocess-exited-with-error
  
  × Building wheel for apriltag (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [75 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      copying python\apriltag.py -> build\lib.win-amd64-cpython-311
      running build_ext
      Traceback (most recent call last):
        File "<string>", line 24, in run
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 466, in check_output
          return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
          with Popen(*popenargs, **kwargs) as process:
               ^^^^

In [6]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import apriltag

ModuleNotFoundError: No module named 'apriltag'

# Vision Input Module

In [7]:
class VisionInput:
    frame = None
    
    def __init__(self, fov, res: tuple):
        self.fov = fov
        self.w = res[0]
        self.h = res[1]
        
        self.cap = cv2.VideoCapture(0)

    #for demo
    def start(self):
        global frame

        while True:
            if not self.cap.isOpened():
                print("failed to init camera")
                exit
            
            cv2.imshow('frame', self.getFrame())

            if (cv2.waitKey(1) == ord('q')):
                break
            
            if (cv2.waitKey(1) == ord('p')):
                print(self.getFrame())
            
        self.cap.release()
        cv2.destroyAllWindows()
    

    def getFrame(self):
        ret, frame = self.cap.read()
        
        if not ret: print('frame malf'); exit
        
        fr = cv2.resize(frame, (self.w, self.h), interpolation=cv2.INTER_AREA)
        return fr
    
    def getGrayFrame(self):
        return cv2.cvtColor(self.getFrame(self), cv2.COLOR_BGR2GRAY)

In [8]:
#TEST BLOCK

vi = VisionInput(70, (1920, 1080))
vi.start()

# Detection Module

In [9]:

class Target:
    def __init__(self, contour, target_type):
        self.contour = contour
        self.target_type = target_type

    def __str__(self):
        return self.target_type + " coords - " + self.contour

    def getType(self):
        return self.target_type

    def getContour(self):
        return self.contour

    def computeYawAngle(R):
        pass

    def get_yaw_degrees(self, result, img):
        FOV_X = 70
        FOV_Y = 70
        #get the 4 points of the april tag
        (ptA, ptB, ptC, ptD) = result.corners
        #find the center point of the tag
        center_tag = ((ptA[0]+ptC[0])/2, (ptA[1]+ptC[1])/2)
        #find the center point of the camera
        center_cam = (len(img[0])/2, len(img)/2)
        #calculate the horizontal difference between the center point of the tag and the point of the camera
        B = center_tag[0] - center_cam[0]
        #calculate horizontal center of camera
        A = len(img[0])/2
        #calculate yaw (look at the diagram below to understand the calculation)
        theta = math.atan(B * math.tan(math.radians(FOV_X / 2)) / A)
        return math.degrees(theta)

    def get_pitch_degrees(self, result, img):
      #WRITE CODE HERE
      FOV_X = 70
      FOV_Y = 70
      #get the 4 points of the april tag
      (ptA, ptB, ptC, ptD) = result.corners
      #find the center point of the tag
      center_tag = ((ptA[0]+ptC[0])/2, (ptA[1]+ptC[1])/2)
      #find the center point of the camera
      center_cam = (len(img[0])/2, len(img)/2)
      #calculate the vertical difference between the center point of the tag and the point of the camera
      B = (ptA[1]+ptC[1])/2 - len(img[0])/2
      #calculate vertical center of camera
      A = len(img)/2
      #calculate pitch
      theta = math.atan(B * math.tan(math.radians(FOV_Y / 2)) / A)
      return math.degrees(theta)

In [10]:
#NOT DONE

class Colored(Target):
    def __init__(self, color):
        self.color = color

    def getType(self):
        return "Colored"
    

In [11]:
#NOT DONE

class AprilTag(Target):
    def __init__(self, id):
        self.id = id
    
    def getType(self):
        return "AprilTag"

    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getAngle(self):
        return self.angle
    
    def getID(self):
        return self.id

In [103]:
class Detector:

    def __init__(self):
        pass
    
    def detectGameElement(self, array, objectsToDetect: list):
        
        eps = 0.015

        frame = cv2.resize(array, (500, int(500*array.shape[0]/array.shape[1])))
        results = dict(zip(objectsToDetect, [None for i in range(len(objectsToDetect))]))

        print(results)


        #CUBE AND CONE DETECTION

        colors = {
            "CUBE": [[158, 255, 255], [90, 50, 70]],
            "CONE": [[25, 255, 255], [22, 50, 70]]
        }
        
        for object in objectsToDetect:
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_frame, np.array(colors[object][1]), np.array(colors[object][0]))

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
            morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)


            if (object == "CUBE"):
                contours, hier = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                x = 0
                y = 0
                w = 0
                h = 0
                for contour in contours:  
                        tx,ty,tw,th = cv2.boundingRect(contour)
                        #print(tx, ty, tw, th)
                        if (tw * th > w * h and not (tx == 0 and ty == 0 and tw == frame.shape[1] and th == frame.shape[0])):
                            x = tx
                            y = ty
                            w = tw
                            h = th

                if (object in objectsToDetect): results[object] = [x, y, w, h]

                #print("X: %2d, Y: %2d, W: %2d, H: %2d" % (x, y, w, h))

                #annotate contour
                cv2.rectangle(frame, (x,y), (x + w, y + h), (0, 0, 255), 2)
                cv2.circle(frame, (int(x + w/2), int(y + h/2)), radius = 0, color = (0, 0, 255), thickness=5)
                cv2.putText(frame, object, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2, cv2.LINE_AA)
                results[object] = Target([x, y, w, h], object)

            if (object == "CONE"):
                straight_contours, hier = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                temp_ctr_max = None
                x = 0
                y = 0
                w = 0
                h = 0

                for contour in straight_contours:
                        tx,ty,tw,th = cv2.boundingRect(contour)
                        #print(tx, ty, tw, th)
                        if (tw * th > w * h and not (tx == 0 and ty == 0 and tw == frame.shape[1] and th == frame.shape[0])):
                            x = tx
                            y = ty
                            w = tw
                            h = th
                            temp_ctr_max = contour

                epsilon = eps * cv2.arcLength(temp_ctr_max, True)
                approx = cv2.approxPolyDP(temp_ctr_max, epsilon, True)
                
                i = 0

                tl = None
                tr = None
                bl = None
                br = None

                for vertex in approx:
                    cv2.circle(frame, vertex[0], 2, (0, 0, 255), 5)

                    x, y = vertex[0]

                    if (tl == None or (x < tl[0] and y < tl[1])):
                        tl = (x, y)
                    elif (tr == None or (x > tr[0] and y < tr[1])):
                        tr = (x, y)
                    elif (bl == None or (x < bl[0] and y > bl[1])):
                        bl = (x, y)
                    elif (br == None or (x > br[0] and y > br[1])):
                        br = (x, y)

                cv2.drawContours(frame, [approx], -1, (0, 0, 255), 2)

                cv2.circle(frame, tl, 2, (0, 255, 0), 5)
                cv2.circle(frame, tr, 2, (0, 255, 0), 5)
                cv2.circle(frame, bl, 2, (0, 255, 0), 5)
                cv2.circle(frame, br, 2, (0, 255, 0), 5)


                #https://docs.revrobotics.com/frc-kickoff-concepts/charged-up-2023/game-elements - dimension uncertainty factored in 
                #NEED TO CHECK IF VERTEX POINTS TL and TR and BL and BR are collinear (this is easy, i am lazy, someone else do it)

                if ((tl[1] - tr[1])/(bl[1] - br[1]) < (45/213) and (tl[1] - tr[1])/(bl[1] - br[1]) < (26/213) and tl[0] > bl[0] and tr[0] < br[0]):
                    results[object] = Target([tl, tr, bl, br], object)


            while True:
                cv2.imshow("o", frame)
                if cv2.waitKey(0):
                    break
            
            cv2.destroyAllWindows()

        return results

    def detectColoredShape(self, array, rgb_col):
        color_dict_HSV = {'black': [[180, 255, 30], [0, 0, 0]],
              'white': [[180, 18, 255], [0, 0, 231]],
              'red1': [[180, 255, 255], [159, 50, 70]],
              'red2': [[9, 255, 255], [0, 50, 70]],
              'green': [[89, 255, 255], [36, 50, 70]],
              'blue': [[128, 255, 255], [90, 50, 70]],
              'yellow': [[35, 255, 255], [25, 50, 70]],
              'purple': [[158, 255, 255], [129, 50, 70]],
              'orange': [[24, 255, 255], [10, 50, 70]],
              'gray': [[180, 18, 230], [0, 0, 40]]}
        
        #================================================================================

        frame = cv2.resize(array, (500, int(500*array.shape[0]/array.shape[1])))
        results = dict(zip(rgb_col, [None for i in range(len(rgb_col))]))

        print(rgb_col)

        for object in rgb_col:
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_frame, np.array(color_dict_HSV[object][1]), np.array(color_dict_HSV[object][0]))

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
            morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
            contours, hier = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            x = 0
            y = 0
            w = 0
            h = 0

            for contour in contours:
                tx,ty,tw,th = cv2.boundingRect(contour)
                print(tx, ty, tw, th)
                if (tw * th > w * h and not (tx == 0 and ty == 0 and tw == frame.shape[1] and th == frame.shape[0])):
                    x = tx
                    y = ty
                    w = tw
                    h = th

            results[object] = [x, y, w, h]

            print("X: %2d, Y: %2d, W: %2d, H: %2d" % (x, y, w, h))

            #annotate contour
            cv2.rectangle(frame, (x,y), (x + w, y + h), (0, 0, 255), 2)
            cv2.circle(frame, (int(x + w/2), int(y + h/2)), radius = 0, color = (0, 0, 255), thickness=5)
            cv2.putText(frame, object, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2, cv2.LINE_AA)

            while True:
                cv2.imshow("o", frame)
                if cv2.waitKey(0):
                    break
            
            cv2.destroyAllWindows()

            results[object] = Target([x, y, w, h], object)

        return results
    


    #will be handled, i assume?
    def detectAprilTag(self, array):
        results = []
        

        
        
        return results



In [106]:
cone = cv2.imread("test-imgs/cone.png")
cube = cv2.imread("test-imgs/cube.png")
at = cv2.imread("test-imgs/test_april_tag.png")

d = Detector()

d.detectGameElement(np.asarray(cone), ["CUBE", "CONE"])

#lmao the colored shape thing sucks, change the hsv values based on what you want + lighting conditions
d.detectColoredShape(np.asarray(cone), ["orange"])

{'CUBE': None, 'CONE': None}
['orange']
486 644 9 1
483 630 9 1
480 617 15 2
419 569 1 1
410 566 1 1
400 561 3 1
390 548 1 1
392 547 1 1
49 523 1 1
92 106 408 560
X: 92, Y: 106, W: 408, H: 560


{'orange': <__main__.Target at 0x23ffc991c10>}

# Alignment Module

# Odometry Module

# Distance Module

In [ ]:
CUBE_TO_CAM = 0 #default
CONE_TO_CAM = 0 #default
class Distance:
  def getDistance(target, result, img):
    if( target == 'Cube'):
      height = CUBE_TO_CAM
    else:
      height = CONE_TO_CAM
    return height/math.tan(np.radians(Detector.get_pitch_degrees(result, img)))